In [1]:
from openai import OpenAI
from nltk.tokenize import BlanklineTokenizer, word_tokenize
import gradio as gr
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string
import re
import numpy as np
from py_files import config
stop = set(stopwords.words('english') + list(string.punctuation))

In [2]:
def jaccard_scorer(set_1, test_list):
    set_2 = set(test_list)
    return float(len(set_1.intersection(set_2))/len(set_1.union(set_2)))

In [3]:
with open('../data/raw/corpus.txt') as f:
    corpus = f.read()
blt = BlanklineTokenizer()
documents = blt.tokenize(corpus)
leman = WordNetLemmatizer()

In [4]:
doc_set = []
for d in documents:
    d = re.sub('[^0-9a-zA-Z]+', ' ', d.lower())
    word_tokens = word_tokenize(d)
    filtered_sentence = [w.lower() for w in word_tokens if not w.lower() in stop]
    filtered_sentence = [leman.lemmatize(w) for w in filtered_sentence]
    doc_set.append(set(filtered_sentence))

In [5]:
client = OpenAI(
    api_key=config.API_KEY,
)

In [6]:
def chat_with_gpt(t_message):
    global messages
    if t_message == 'new conversation':
        messages = [messages[0]]
    elif t_message == 'messages':
        pass
    else:
        message = f"User : {t_message}"
        if message:
            messages.append(
                {"role": "user", "content": message},
            )
            chat = client.chat.completions.create(model="gpt-3.5-turbo",
                                                  messages=messages,
                                                 temperature = 0.0)
        reply = chat.choices[0].message.content
        return reply

In [7]:
#Some prompts for testing
prompts = ['''Something is wrong with the on screen timer.''', 
          '''I just spent 2 hours reading the instructions. Do i get paid for this''',
          '''can i prompt the model to generate a file''', 
          '''do i penalize the model for not using the .head function''',
           '''do i rate the edited response or the original''',
           '''is “Sorry we couldn't finish the response at this time. Please try again later.” considered a canned response''',
          ]

In [8]:
internal_idxs = []
messages = [{'role': 'system', 'content': 'you are a helpful question answering bot. Your job is to provide \
friendly advice regarding questions you are asked. You will receive a question and up to 3 rules to guide you. however, \
the rules may not apply to the question asked. for each rule, you are to think about the question and the rule and \
decide if the rule applies. if the rule does not apply, ignore the rule. otherwise think about how the rule\
applies to answer the prompt. i want your response to include the following information: \
tell the user you are a friendly bot, \
paraphrase the question \
provide the answer based on the rules, but do not quote the rules, \
your answer must be based on one of the rules"'
            }]

#This is virtually a copy of the method used to get the doc set,Abstract Abstract, Abstract
def get_test_set(text):
    d = re.sub('[^0-9a-zA-Z]+', ' ', text.lower())
    word_tokens = word_tokenize(d)
    filtered_sentence = [w.lower() for w in word_tokens if not w.lower() in stop]
    filtered_sentence = [leman.lemmatize(w) for w in filtered_sentence]
    return set(filtered_sentence)


def get_rules_idxs(t_docs_set, test_set):
    j_scores = []
    for t_d in t_docs_set:
        j_scores.append(jaccard_scorer(t_d, test_set))
    idxs = np.argpartition(np.array(j_scores), -3)[-3:]
    idxs = [i for i in idxs if j_scores[i] != 0]
    return idxs

def append_rules(idxs, t_prompts):
    for i, rule_idx in enumerate(idxs):
        t_prompts += f'\nRule {i+1}: {documents[rule_idx]}'
    return t_prompts

def main(text):
    test_set = get_test_set(text)
    idxs = get_rules_idxs(doc_set, test_set)
    idxs = list(reversed(idxs))
    internal_idxs.append(idxs)
    prompt_w_rules = append_rules(idxs, text)
    print(prompt_w_rules)
    return chat_with_gpt(f'answer the following question, use the rules given as a guideline: {prompt_w_rules}')

prompt = prompts[5]
print(main(prompt))

is “Sorry we couldn't finish the response at this time. Please try again later.” considered a canned response
Rule 1: Responses that end with “Sorry we couldn't finish the response at this time. Please try again later.” are not considered canned responses and are not unratable. The lone exception to this is if there is nothing else in the response (meaning literally no code/output and no other text).
Rule 2: Please do not select error messages as the better response. If you receive error messages from both models, and editing is not working, use the checkbox and submit the conversation. Do not try to move forward with an erroneous response.
Rule 3: You can use files that others have created, but please try to create a couple of your own first.
I am a friendly bot here to help you. The question you asked is whether the response "Sorry we couldn't finish the response at this time. Please try again later." is considered a canned response. Based on the rules provided, the answer is that th